In [1]:
import os
import sys

### Creating Paths

In [2]:
WORKSPACE_PATH = 'Tensorflow\\workspace'
SCRIPTS_PATH = 'Tensorflow\\scripts'
APIMODEL_PATH = 'Tensorflow\\models'
ANNOTATION_PATH = WORKSPACE_PATH+'\\annotations'
IMAGE_PATH = WORKSPACE_PATH+'\\images'
MODEL_PATH = WORKSPACE_PATH+'\\models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'\\pre-trained-models'
CONFIG_PATH = MODEL_PATH+'\\my_ssd_mobnet\\pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'\\my_ssd_mobnet\\'

### Classes dictionary

In [3]:
class_label_path = os.path.join(os.getcwd(), WORKSPACE_PATH, 'labelled_class_list.txt')
with open(class_label_path, 'r') as f:
    class_list = [line.rstrip() for line in f]
class_labels_dict_list = [{'name':cls.lower(),'id':idx+1} for idx,cls in enumerate(class_list)]

class_labels_dict_list

[{'name': 'bicycle', 'id': 1},
 {'name': 'bike', 'id': 2},
 {'name': 'bus', 'id': 3},
 {'name': 'car', 'id': 4},
 {'name': 'cng', 'id': 5},
 {'name': 'truck', 'id': 6}]

In [4]:
#TFOD requires pbtxt formate class file

with open(ANNOTATION_PATH + '\\label_map.pbtxt', 'w') as f:
    for label in class_labels_dict_list:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [5]:
!python {SCRIPTS_PATH + '\\generate_tfrecord.py'} -x {IMAGE_PATH + '\\train'} -l {ANNOTATION_PATH + '\\label_map.pbtxt'} -o {ANNOTATION_PATH + '\\train.record'}
!python {SCRIPTS_PATH + '\\generate_tfrecord.py'} -x{IMAGE_PATH + '\\test'} -l {ANNOTATION_PATH + '\\label_map.pbtxt'} -o {ANNOTATION_PATH + '\\test.record'}
    

{'bicycle': 1, 'bike': 2, 'bus': 3, 'car': 4, 'cng': 5, 'truck': 6}
Successfully created the TFRecord file: Tensorflow\workspace\annotations\train.record
{'bicycle': 1, 'bike': 2, 'bus': 3, 'car': 4, 'cng': 5, 'truck': 6}
Successfully created the TFRecord file: Tensorflow\workspace\annotations\test.record


### Copy Model Config to Training Folder

In [6]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

!mkdir {'Tensorflow\\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'\\ssd_resnet101_v1_fpn_640x640_coco17_tpu-8\\pipeline.config'} {MODEL_PATH+'\\'+CUSTOM_MODEL_NAME}

A subdirectory or file Tensorflow\workspace\models\my_ssd_mobnet already exists.
'cp' is not recognized as an internal or external command,
operable program or batch file.


### Update Config For Transfer Learning

In [7]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format


In [10]:
CONFIG_PATH = MODEL_PATH+'\\'+CUSTOM_MODEL_NAME+'\\pipeline.config'
print(CONFIG_PATH)
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config


In [11]:
config

{'model': ssd {
   num_classes: 6
   image_resizer {
     fixed_shape_resizer {
       height: 640
       width: 640
     }
   }
   feature_extractor {
     type: "ssd_resnet101_v1_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 0.00039999998989515007
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5


In [12]:
# Over writting config file for less classes and type as 'detection' instead of 'classification
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [13]:
pipeline_config.model.ssd.num_classes = 6
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [14]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 

### Train Model

In [29]:
print("""python {}\\research\\object_detection\\model_main_tf2.py --model_dir={}\\{} --pipeline_config_path={}\\{}\\pipeline.config --num_train_steps=5""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python Tensorflow\models\research\object_detection\model_main_tf2.py --model_dir=Tensorflow\workspace\models\my_ssd_mobnet --pipeline_config_path=Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config --num_train_steps=5
